In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
assert PROJECT_JSON["signal_type"] == "context"

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = PROJECT_JSON["sample_alias"]

In [ ]:
for element_type, dataframe in (
    ("feature", feature_x_sample_processed),
    ("sample", feature_x_sample_processed.T),
):

    if element_type not in PROJECT_JSON["context_element_types"]:

        continue

    element_x_fit_parameter = pd.read_csv(
        PATH["{}_x_fit_parameter.tsv".format(element_type)], sep="\t", index_col=0
    )

    context_matrix = kraft.make_context_matrix(
        dataframe,
        n_job=PROJECT_JSON["n_job"],
        skew_t_pdf_fit_parameter=element_x_fit_parameter,
        output_file_path=PATH["feature_x_sample.{}_context.tsv".format(element_type)],
    )

    context_matrix = pd.read_csv(
        PATH["feature_x_sample.{}_context.tsv".format(element_type)],
        sep="\t",
        index_col=0,
    )

    kraft.summarize_feature_x_sample(
        context_matrix,
        feature_x_sample_alias="{} Context".format(
            PROJECT_JSON["{}_alias".format(element_type)]
        ),
        feature_x_sample_value_name="Context",
        plot_heat_map_max_size=PROJECT_JSON["plot_heat_map_max_size"],
        plot_histogram_max_size=PROJECT_JSON["plot_histogram_max_size"],
        plot_rug_max_size=PROJECT_JSON["plot_rug_max_size"],
    )

    for context_type, clip_keyword_arguments in (
        ("negative", {"upper": 0}),
        ("positive", {"lower": 0}),
    ):

        element_context_sorted = (
            context_matrix.clip(**clip_keyword_arguments).sum(axis=1).sort_values()
        )

        elements_to_peek = (
            element_context_sorted.index
            & PROJECT_JSON["{}s_to_peek".format(element_type)]
        )

        kraft.plot_plotly_figure(
            {
                "layout": {
                    "title": {"text": PROJECT_JSON["{}_alias".format(element_type)]},
                    "xaxis": {"title": "Rank"},
                    "yaxis": {"title": "{} Context Sum".format(context_type.title())},
                    "annotations": [
                        {
                            "x": element,
                            "y": element_context_sorted[element],
                            "text": element,
                            "textangle": -90,
                            "font": {"size": 10},
                            "arrowhead": 2,
                            "arrowsize": 0.64,
                        }
                        for element in elements_to_peek
                    ],
                },
                "data": [
                    {
                        "type": "scatter",
                        "name": "All",
                        "x": element_context_sorted.index,
                        "y": element_context_sorted,
                        "text": element_context_sorted.index,
                    }
                ],
            },
            None,
        )

        if context_type == "negative":

            extreme_elements = element_context_sorted.index[:3]

        elif context_type == "positive":

            extreme_elements = element_context_sorted.index[-3:]

        for extreme_element in extreme_elements:

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                extreme_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            kraft.plot_context(
                dataframe.loc[extreme_element],
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                xaxis_title=PROJECT_JSON["feature_x_sample_value_name"],
            )